In [1]:
import pandas as pd
import csv
import re
import statsmodels.api as sm


from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score, root
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
file_url = '/Users/xzhou/github/project_archives/files_airbnb/{}/calendar.csv'

sf_file = file_url.format('sf_airbnb')
ok_file = file_url.format('ok_airbnb')
sc_file = file_url.format('sc_airbnb')
scz_file = file_url.format('scz_airbnb')

In [3]:
pricing_sf = pd.read_csv(sf_file, parse_dates=['date'])
pricing_ok = pd.read_csv(ok_file, parse_dates=['date'])
pricing_sc = pd.read_csv(sc_file, parse_dates=['date'])
pricing_scz = pd.read_csv(scz_file, parse_dates=['date'])

In [4]:
print(pricing_sf.shape, pricing_ok.shape, pricing_sc.shape, pricing_scz.shape)

(2420680, 4) (1057770, 4) (2068820, 4) (573050, 4)


In [5]:
frames = [pricing_sf, pricing_ok, pricing_sc, pricing_scz]

In [6]:
df_pricing = pd.concat(frames, keys=['sf', 'ok', 'sc', 'scz'])

In [7]:
df_pricing.listing_id.nunique()

16768

In [8]:
df_pricing.head(100)

listing_id       date available    price
sf 0          958 2019-08-05         f      NaN
   1          958 2019-08-04         f      NaN
   2          958 2019-08-03         f      NaN
   3          958 2019-08-02         f      NaN
   4          958 2019-08-01         f      NaN
   5          958 2019-07-31         f      NaN
   6          958 2019-07-30         f      NaN
   7          958 2019-07-29         f      NaN
   8          958 2019-07-28         f      NaN
   9          958 2019-07-27         f      NaN
   10         958 2019-07-26         f      NaN
   11         958 2019-07-25         f      NaN
   12         958 2019-07-24         f      NaN
   13         958 2019-07-23         f      NaN
   14         958 2019-07-22         f      NaN
   15         958 2019-07-21         f      NaN
   16         958 2019-07-20         f      NaN
   17         958 2019-07-19         f      NaN
   18         958 2019-07-18         f      NaN
   19         958 2019-07-17         f      NaN
   20         958 2019-07-16         f      NaN
   21         958 2019-07-15         f      NaN
   22         958 2019-07-14         f      NaN
   23         958 2019-07-13         f      NaN
   24         958 2019-07-12         f      NaN
   25         958 2019-06-09         f      NaN
   26         958 2019-06-08         f      NaN
   27         958 2019-06-07         f      NaN
   28         958 2019-06-06         f      NaN
   29         958 2019-06-05         f      NaN
...           ...        ...       ...      ...
   70         958 2019-04-25         t  $181.00
   71         958 2019-04-24         t  $181.00
   72         958 2019-04-23         t  $181.00
   73         958 2019-04-22         t  $181.00
   74         958 2019-04-21         t  $181.00
   75         958 2019-04-20         t  $191.00
   76         958 2019-04-19         t  $191.00
   77         958 2019-04-18         t  $181.00
   78         958 2019-04-17         t  $181.00
   79         958 2019-04-16         t  $181.00
   80         958 2019-04-15         t  $181.00
   81         958 2019-04-14         t  $181.00
   82         958 2019-04-13         t  $191.00
   83         958 2019-04-12         t  $191.00
   84         958 2019-04-11         t  $181.00
   85         958 2019-04-10         t  $181.00
   86         958 2019-04-09         t  $181.00
   87         958 2019-04-08         t  $181.00
   88         958 2019-04-07         t  $181.00
   89         958 2019-04-06         t  $191.00
   90         958 2019-04-05         t  $191.00
   91         958 2019-04-04         t  $181.00
   92         958 2019-04-03         t  $181.00
   93         958 2019-04-02         t  $181.00
   94         958 2019-04-01         t  $181.00
   95         958 2019-03-31         t  $181.00
   96         958 2019-03-30         t  $191.00
   97         958 2019-03-29         t  $191.00
   98         958 2019-03-28         t  $181.00
   99         958 2019-03-27         t  $181.00

[100 rows x 4 columns]

In [9]:
df_test = df_pricing[df_pricing['available']=='f']

In [10]:
df_pricing_smaller = df_pricing[df_pricing['available']=='t']

In [11]:
df_pricing_smaller.shape

(2853357, 4)

In [12]:
df_pricing_smaller.listing_id.nunique()

14770

In [13]:
df_pricing_smaller.date.describe(include='all')

count                 2853357
unique                    389
top       2018-11-03 00:00:00
freq                    10393
first     2018-08-06 00:00:00
last      2019-08-29 00:00:00
Name: date, dtype: object

In [14]:
df_pricing_smaller.columns

Index(['listing_id', 'date', 'available', 'price'], dtype='object')

In [15]:
def currency_to_float(string):
    """
    Convert currency to float
    """
    float_ = float(str(string).strip('$').replace(',',''))
    return float_

In [16]:
# Convert price to float format

df_pricing_smaller['price'] = df_pricing_smaller['price'].apply(lambda x: currency_to_float(x))

/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
# Add a month column based on date

df_pricing_smaller['month'] = df_pricing_smaller['date'].dt.month

/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
# Add a day_of_week column based on Date. Use default format: Monday=0, Sunday=6

df_pricing_smaller['day_of_week'] = df_pricing_smaller['date'].dt.weekday

/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
df_pricing_smaller.head()

listing_id       date available  price  month  day_of_week
sf 63         958 2019-05-02         t  181.0      5            3
   64         958 2019-05-01         t  181.0      5            2
   65         958 2019-04-30         t  181.0      4            1
   66         958 2019-04-29         t  181.0      4            0
   67         958 2019-04-28         t  181.0      4            6

In [20]:
df_pricing_smaller.shape

(2853357, 6)

In [21]:
df_pricing_smaller.describe(include ='all')

,listing_id,date,available,price,month,day_of_week
count,2.853357e+06,2853357,2853357,2.853357e+06,2.853357e+06,2.853357e+06
unique,NaN,389,1,NaN,NaN,NaN
top,NaN,2018-11-03 00:00:00,t,NaN,NaN,NaN
freq,NaN,10393,2853357,NaN,NaN,NaN
first,NaN,2018-08-06 00:00:00,NaN,NaN,NaN,NaN
last,NaN,2019-08-29 00:00:00,NaN,NaN,NaN,NaN
mean,1.405523e+07,NaN,NaN,2.132333e+02,6.571592e+00,2.988646e+00
std,8.563144e+06,NaN,NaN,7.035042e+02,3.629179e+00,2.003061e+00
min,9.580000e+02,NaN,NaN,1.000000e+01,1.000000e+00,0.000000e+00
25%,6.453774e+06,NaN,NaN,8.000000e+01,3.000000e+00,1.000000e+00


In [23]:
df_pricing_smaller.loc['sf'].describe(include='all')

,listing_id,date,available,price,month,day_of_week
count,1.073584e+06,1073584,1073584,1.073584e+06,1.073584e+06,1.073584e+06
unique,NaN,365,1,NaN,NaN,NaN
top,NaN,2018-11-03 00:00:00,t,NaN,NaN,NaN
freq,NaN,3953,1073584,NaN,NaN,NaN
first,NaN,2018-08-06 00:00:00,NaN,NaN,NaN,NaN
last,NaN,2019-08-05 00:00:00,NaN,NaN,NaN,NaN
mean,1.322873e+07,NaN,NaN,2.277898e+02,6.511572e+00,2.996720e+00
std,9.057012e+06,NaN,NaN,1.061485e+03,3.633400e+00,2.003876e+00
min,9.580000e+02,NaN,NaN,1.000000e+01,1.000000e+00,0.000000e+00
25%,4.097700e+06,NaN,NaN,1.000000e+02,3.000000e+00,1.000000e+00


In [24]:
df_pricing_smaller.loc['ok'].describe(include='all')

,listing_id,date,available,price,month,day_of_week
count,3.375010e+05,337501,337501,337501.00000,337501.000000,337501.000000
unique,NaN,365,1,NaN,NaN,NaN
top,NaN,2018-11-12 00:00:00,t,NaN,NaN,NaN
freq,NaN,1415,337501,NaN,NaN,NaN
first,NaN,2018-08-16 00:00:00,NaN,NaN,NaN,NaN
last,NaN,2019-08-15 00:00:00,NaN,NaN,NaN,NaN
mean,1.319126e+07,NaN,NaN,140.81283,6.635752,2.987301
std,8.759368e+06,NaN,NaN,196.83452,3.644244,2.002487
min,3.083000e+03,NaN,NaN,20.00000,1.000000,0.000000
25%,4.777769e+06,NaN,NaN,69.00000,3.000000,1.000000


In [25]:
df_pricing_smaller.loc['sc'].describe(include='all')

,listing_id,date,available,price,month,day_of_week
count,1.177267e+06,1177267,1177267,1.177267e+06,1.177267e+06,1.177267e+06
unique,NaN,365,1,NaN,NaN,NaN
top,NaN,2018-11-09 00:00:00,t,NaN,NaN,NaN
freq,NaN,4336,1177267,NaN,NaN,NaN
first,NaN,2018-08-14 00:00:00,NaN,NaN,NaN,NaN
last,NaN,2019-08-13 00:00:00,NaN,NaN,NaN,NaN
mean,1.520984e+07,NaN,NaN,2.039792e+02,6.635694e+00,2.991873e+00
std,7.909631e+06,NaN,NaN,3.778313e+02,3.619311e+00,2.002146e+00
min,1.146400e+04,NaN,NaN,1.800000e+01,1.000000e+00,0.000000e+00
25%,9.029655e+06,NaN,NaN,6.600000e+01,3.000000e+00,1.000000e+00


In [26]:
# Count how many months each house was listed in the last year

df_month_count = df_pricing_smaller.groupby(['listing_id']).month.nunique().reset_index()
df_month_count.rename(columns={'month':'month_count'}, inplace=True)
df_month_count.head()

,listing_id,month_count
0,958,10
1,3083,5
2,3264,3
3,5739,2
4,5858,12


In [27]:
df_month_count.shape

(14770, 2)

In [28]:
# Count how many days of week each house was listed in the last year

df_day_of_week_count = df_pricing_smaller.groupby(['listing_id']).day_of_week.nunique().reset_index()
df_day_of_week_count.rename(columns={'day_of_week':'day_of_week_count'}, inplace=True)
df_day_of_week_count.head()

,listing_id,day_of_week_count
0,958,7
1,3083,7
2,3264,7
3,5739,7
4,5858,7


In [29]:
df_day_of_week_count.shape

(14770, 2)

In [30]:
df_pricing_with_counts = pd.merge(left=df_pricing_smaller, 
                                  right=df_month_count,
                                  how='left', 
                                  left_on='listing_id', 
                                  right_on='listing_id')
df_pricing_with_counts.head()

,listing_id,date,available,price,month,day_of_week,month_count
0,958,2019-05-02,t,181.0,5,3,10
1,958,2019-05-01,t,181.0,5,2,10
2,958,2019-04-30,t,181.0,4,1,10
3,958,2019-04-29,t,181.0,4,0,10
4,958,2019-04-28,t,181.0,4,6,10


In [31]:
df_pricing_with_counts.shape

(2853357, 7)

In [32]:
df_pricing_with_counts = pd.merge(left=df_pricing_with_counts, 
                                  right=df_day_of_week_count,
                                  how='left', 
                                  left_on='listing_id', 
                                  right_on='listing_id')
df_pricing_with_counts.head()

,listing_id,date,available,price,month,day_of_week,month_count,day_of_week_count
0,958,2019-05-02,t,181.0,5,3,10,7
1,958,2019-05-01,t,181.0,5,2,10,7
2,958,2019-04-30,t,181.0,4,1,10,7
3,958,2019-04-29,t,181.0,4,0,10,7
4,958,2019-04-28,t,181.0,4,6,10,7


In [33]:
df_pricing_with_counts.shape

(2853357, 8)

In [34]:
df_pricing_with_counts.describe(include='all')

,listing_id,date,available,price,month,day_of_week,month_count,day_of_week_count
count,2.853357e+06,2853357,2853357,2.853357e+06,2.853357e+06,2.853357e+06,2.853357e+06,2.853357e+06
unique,NaN,389,1,NaN,NaN,NaN,NaN,NaN
top,NaN,2018-11-03 00:00:00,t,NaN,NaN,NaN,NaN,NaN
freq,NaN,10393,2853357,NaN,NaN,NaN,NaN,NaN
first,NaN,2018-08-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,2019-08-29 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
mean,1.405523e+07,NaN,NaN,2.132333e+02,6.571592e+00,2.988646e+00,1.020866e+01,6.995392e+00
std,8.563144e+06,NaN,NaN,7.035042e+02,3.629179e+00,2.003061e+00,2.696574e+00,1.206497e-01
min,9.580000e+02,NaN,NaN,1.000000e+01,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
25%,6.453774e+06,NaN,NaN,8.000000e+01,3.000000e+00,1.000000e+00,9.000000e+00,7.000000e+00


In [43]:
df_pricing_with_counts.price.describe(percentiles=[0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95])

count    2.581280e+06
mean     1.603721e+02
std      1.016462e+02
min      4.000000e+01
5%       5.000000e+01
10%      6.000000e+01
25%      8.400000e+01
50%      1.310000e+02
75%      2.050000e+02
90%      3.020000e+02
95%      3.840000e+02
max      5.000000e+02
Name: price, dtype: float64

In [46]:
# Use 5% and 95% percentiles as cutoff

df_pricing_with_counts = df_pricing_with_counts[(df_pricing_with_counts.price>=50)
                                                & (df_pricing_with_counts.price<=400)]

In [47]:
df_pricing_with_counts.shape

(2377562, 8)

In [48]:
# (df_pricing_with_counts['month_count']==12)
#                   & (df_pricing_with_counts['day_of_week_count']==7)

df_pricing_available = df_pricing_with_counts[(df_pricing_with_counts['month_count']==12)
                                 & (df_pricing_with_counts['day_of_week_count']==7)]

In [49]:
df_pricing_available.shape

(1362356, 8)

In [50]:
df_pricing_available.listing_id.nunique()

4253

In [51]:
df_pricing_available.isnull().sum()

listing_id           0
date                 0
available            0
price                0
month                0
day_of_week          0
month_count          0
day_of_week_count    0
dtype: int64

In [53]:
df_pricing_available.describe(percentiles=[0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95])

,listing_id,price,month,day_of_week,month_count,day_of_week_count
count,1.362356e+06,1.362356e+06,1.362356e+06,1.362356e+06,1362356.0,1362356.0
mean,1.350360e+07,1.578347e+02,6.318691e+00,2.986385e+00,12.0,7.0
std,8.753801e+06,8.673772e+01,3.496964e+00,2.005098e+00,0.0,0.0
min,5.858000e+03,5.000000e+01,1.000000e+00,0.000000e+00,12.0,7.0
5%,5.657012e+05,5.900000e+01,1.000000e+00,0.000000e+00,12.0,7.0
10%,1.299242e+06,6.500000e+01,2.000000e+00,0.000000e+00,12.0,7.0
25%,5.406477e+06,8.900000e+01,3.000000e+00,1.000000e+00,12.0,7.0
50%,1.373254e+07,1.360000e+02,6.000000e+00,3.000000e+00,12.0,7.0
75%,2.139994e+07,2.000000e+02,1.000000e+01,5.000000e+00,12.0,7.0
90%,2.537085e+07,2.950000e+02,1.100000e+01,6.000000e+00,12.0,7.0


In [56]:
df_pricing_by_month_weekday = df_pricing_available.groupby(['listing_id', 'month', 'day_of_week']) \
                              .price.mean()  \
                              .reset_index()
df_pricing_by_month_weekday.head()

,listing_id,month,day_of_week,price
0,5858,1,0,235.0
1,5858,1,1,235.0
2,5858,1,2,235.0
3,5858,1,3,235.0
4,5858,1,4,235.0


In [57]:
df_pricing_by_month_weekday.describe(include='all')

,listing_id,month,day_of_week,price
count,3.384600e+05,338460.000000,338460.000000,338460.000000
mean,1.342273e+07,6.453646,3.001971,158.217563
std,8.750020e+06,3.464062,2.003586,86.813187
min,5.858000e+03,1.000000,0.000000,50.000000
25%,5.286782e+06,3.000000,1.000000,89.000000
50%,1.365649e+07,6.000000,3.000000,137.000000
75%,2.129177e+07,10.000000,5.000000,202.000000
max,2.815247e+07,12.000000,6.000000,400.000000


In [58]:
df_pricing_by_month_weekday.isnull().sum()

listing_id     0
month          0
day_of_week    0
price          0
dtype: int64

In [60]:
df_pricing_by_month_weekday.rename(columns={'price':'price_by_month_week'}, inplace=True)

In [61]:
df_pricing_by_month_weekday.shape

(338460, 4)

In [62]:
df_pricing_final = pd.get_dummies(df_pricing_by_month_weekday, columns=['month', 'day_of_week'])

In [63]:
df_pricing_final.columns

Index(['listing_id', 'price_by_month_week', 'month_1', 'month_2', 'month_3',
       'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9',
       'month_10', 'month_11', 'month_12', 'day_of_week_0', 'day_of_week_1',
       'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'day_of_week_5',
       'day_of_week_6'],
      dtype='object')

In [64]:
df_pricing_final.shape

(338460, 21)

In [68]:
pkl_file = '/Users/xzhou/github/project_archives/files_airbnb/{}.csv'

pkl = pkl_file.format('all_listings')

df_listings_reviews = pd.read_pickle(pkl)
df_listings_reviews.head()

,id,host_response_rate,host_listings_count,accommodates,bathrooms,bedrooms,beds,price,security_deposit,cleaning_fee,...,instant_bookable_f,is_business_travel_ready_f,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict,cancellation_policy_strict_14_with_grace_period,cancellation_policy_super_strict_30,cancellation_policy_super_strict_60,require_guest_profile_picture_f,require_guest_phone_verification_f
0,958,0.92,1,3,1.0,1.0,2.0,170.0,1,2.00000,...,1,1,0,1,0,0,0,0,1,1
1,5858,1.00,2,5,1.0,2.0,3.0,235.0,1,2.00000,...,1,1,0,0,0,1,0,0,1,1
2,7918,1.00,10,2,4.0,1.0,1.0,65.0,1,1.69897,...,1,1,0,0,0,1,0,0,1,1
3,8142,1.00,10,2,4.0,1.0,1.0,65.0,1,1.69897,...,1,1,0,0,0,1,0,0,1,1
5,8567,0.80,1,6,1.0,2.0,3.0,255.0,0,2.09691,...,1,1,0,1,0,0,0,0,1,1


In [69]:
df_listings_reviews.shape

(11700, 336)

In [76]:
df_final = pd.merge(left=df_pricing_final, 
                    right=df_listings_reviews,
                    how='inner',
                    left_on='listing_id',
                    right_on='id')

In [77]:
df_final.shape

(260392, 357)

In [78]:
df_final.isnull().sum()

listing_id                                         0
price_by_month_week                                0
month_1                                            0
month_2                                            0
month_3                                            0
month_4                                            0
month_5                                            0
month_6                                            0
month_7                                            0
month_8                                            0
month_9                                            0
month_10                                           0
month_11                                           0
month_12                                           0
day_of_week_0                                      0
day_of_week_1                                      0
day_of_week_2                                      0
day_of_week_3                                      0
day_of_week_4                                 

In [80]:
df_final.shape

(260392, 357)

In [81]:
df_final.columns

Index(['listing_id', 'price_by_month_week', 'month_1', 'month_2', 'month_3',
       'month_4', 'month_5', 'month_6', 'month_7', 'month_8',
       ...
       'instant_bookable_f', 'is_business_travel_ready_f',
       'cancellation_policy_flexible', 'cancellation_policy_moderate',
       'cancellation_policy_strict',
       'cancellation_policy_strict_14_with_grace_period',
       'cancellation_policy_super_strict_30',
       'cancellation_policy_super_strict_60',
       'require_guest_profile_picture_f',
       'require_guest_phone_verification_f'],
      dtype='object', length=357)

In [82]:
df_final.drop(['id', 'price'], axis=1, inplace=True)

In [83]:
df_final.shape

(260392, 355)

In [84]:
# Create correlation matrix
corr_matrix = df_final.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.90
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]

to_drop

In [85]:
to_drop

['review_scores_value_log10']

In [87]:
# Drop features that have high correlation

df_final.drop(to_drop, axis=1, inplace=True)

In [88]:
df_final.shape

(260392, 354)

In [89]:
# word count

X = df_final.drop(['listing_id', 'price_by_month_week'], axis=1)
y = df_final['price_by_month_week']

In [90]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.3, random_state=42)

In [91]:
ss = StandardScaler()
X_train_trans = ss.fit_transform(X_train)
X_test_trans = ss.transform (X_test)

In [92]:
X_train_trans.shape

(182274, 352)

In [93]:
X_test_trans.shape

(78118, 352)

In [94]:
lr = linear_model.LinearRegression()
lr.fit(X_train_trans, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [95]:
lr.score(X_test_trans, y_test)

0.695006850955054

In [96]:
y_predict = lr.predict(X_test_trans)

In [97]:
r2_score(y_predict, y_test)

0.5618587800155335

In [98]:
mean_squared_error(y_predict, y_test)

2248.8278807217257

In [99]:
model=sm.OLS(y_train, sm.add_constant(X_train_trans))
fit=model.fit()
fit.summary()

/Users/xzhou/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1100: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/Users/xzhou/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/xzhou/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/xzhou/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                            
===============================================================================
Dep. Variable:     price_by_month_week   R-squared:                       0.696
Model:                             OLS   Adj. R-squared:                  0.695
Method:                  Least Squares   F-statistic:                     1474.
Date:                 Sun, 16 Sep 2018   Prob (F-statistic):               0.00
Time:                         16:23:46   Log-Likelihood:            -9.6181e+05
No. Observations:               182274   AIC:                         1.924e+06
Df Residuals:                   181991   BIC:                         1.927e+06
Df Model:                          282                                         
Covariance Type:             nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        153.2869      0.112   1373.115      0.000     153.068     153.506
x1         -3.555e+11   4.67e+11     -0.761      0.447   -1.27e+12     5.6e+11
x2         -3.565e+11   4.68e+11     -0.761      0.447   -1.27e+12    5.62e+11
x3         -3.562e+11   4.68e+11     -0.761      0.447   -1.27e+12    5.61e+11
x4         -3.557e+11   4.67e+11     -0.761      0.447   -1.27e+12    5.61e+11
x5         -3.557e+11   4.67e+11     -0.761      0.447   -1.27e+12    5.61e+11
x6         -3.555e+11   4.67e+11     -0.761      0.447   -1.27e+12     5.6e+11
x7         -3.552e+11   4.67e+11     -0.761      0.447   -1.27e+12     5.6e+11
x8         -3.482e+11   4.58e+11     -0.761      0.447   -1.25e+12    5.49e+11
x9         -3.311e+11   4.35e+11     -0.761      0.447   -1.18e+12    5.22e+11
x10        -3.492e+11   4.59e+11     -0.761      0.447   -1.25e+12     5.5e+11
x11        -3.522e+11   4.63e+11     -0.761      0.447   -1.26e+12    5.55e+11
x12        -3.555e+11   4.67e+11     -0.761      0.447   -1.27e+12     5.6e+11
x13        -1.195e+12   4.36e+12     -0.274      0.784   -9.73e+12    7.34e+12
x14        -1.186e+12   4.32e+12     -0.274      0.784   -9.66e+12    7.29e+12
x15        -1.185e+12   4.32e+12     -0.274      0.784   -9.65e+12    7.28e+12
x16        -1.194e+12   4.35e+12     -0.274      0.784   -9.72e+12    7.34e+12
x17         -1.19e+12   4.34e+12     -0.274      0.784   -9.69e+12    7.31e+12
x18        -1.186e+12   4.32e+12     -0.274      0.784   -9.66e+12    7.29e+12
x19        -1.198e+12   4.37e+12     -0.274      0.784   -9.76e+12    7.36e+12
x20           -0.5054      0.230     -2.201      0.028      -0.955      -0.055
x21            4.1483      0.186     22.337      0.000       3.784       4.512
x22           24.7055      0.274     90.184      0.000      24.169      25.242
x23           -0.2644      0.145     -1.821      0.069      -0.549       0.020
x24           22.4919      0.194    115.984      0.000      22.112      22.872
x25           -8.7038      0.242    -35.988      0.000      -9.178      -8.230
x26           -0.9219      0.126     -7.339      0.000      -1.168      -0.676
x27            3.8988      0.151     25.889      0.000       3.604       4.194
x28            6.7421      0.148     45.698      0.000       6.453       7.031
x29           -3.2063      0.140    -22.907      0.000      -3.481      -2.932
x30           -2.6831      0.229    -11.712      0.000      -3.132      -2.234
x31          -10.3067      0.192    -53.683      0.000     -10.683      -9.930
x32           -8.3149      0.216    -38.547      0.000      -8.738      -7.892
x33            3.6323      0.156     23.318      0.000       3.327       3.938
x34           -0.1900      0.242     -0.786      0.432      -0.664       0.284
x35            1.5240      0.140     10.921      0.000       1.250    

In [101]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(n_estimators=100, 
                               criterion='mse', 
                               random_state=3, 
                               n_jobs=-1)
forest.fit(X_train_trans, y_train)
y_train_pred = forest.predict(X_train_trans)
y_test_pred = forest.predict(X_test_trans)

print('MSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
print('R^2 train: %.3f, test: %.3f' % (
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))

MSE train: 4.032, test: 26.640
R^2 train: 0.999, test: 0.996


In [103]:
feature_labels = X.columns

In [104]:
feature_labels

Index(['month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10',
       ...
       'instant_bookable_f', 'is_business_travel_ready_f',
       'cancellation_policy_flexible', 'cancellation_policy_moderate',
       'cancellation_policy_strict',
       'cancellation_policy_strict_14_with_grace_period',
       'cancellation_policy_super_strict_30',
       'cancellation_policy_super_strict_60',
       'require_guest_profile_picture_f',
       'require_guest_phone_verification_f'],
      dtype='object', length=352)

In [115]:
feature_tuples = zip (feature_labels, forest.feature_importances_)
features_list = [list(feature_tuple) for feature_tuple in feature_tuples]

In [116]:
features_list.sort(key=lambda x: abs(x[1]), reverse=True)

In [117]:
features_list

[['bedrooms', 0.3779611219774654],
 ['room_type_Entire home/apt', 0.08621581584273132],
 ['cleaning_fee', 0.05495123226541247],
 ['accommodates', 0.05008309977841667],
 ['host_listings_count', 0.04106299177196036],
 ['reviews_per_month', 0.03560719197434737],
 ['number_of_reviews', 0.021142160229082926],
 ['review_scores_rating_log10', 0.016351073166625863],
 ['calculated_host_listings_count', 0.01564030509043598],
 ['city_Oakland', 0.014027913273177421],
 ['host_duration', 0.013569153353121963],
 ['city_San Francisco', 0.012945384417496772],
 ['beds', 0.010727598449813802],
 ['bathrooms', 0.009965611501310395],
 ['guests_included', 0.009112864784728539],
 ['host_response_rate', 0.0089809295887684],
 ['yrs_since_first_review', 0.008191202472606168],
 ['Air conditioning', 0.006680119223938971],
 ['Self checkin', 0.00663697762074117],
 ['host_is_superhost_f', 0.006129690121940262],
 ['review_scores_location_log10', 0.005442357654315655],
 ['neighbourhood_cleansed_Downtown/Civic Center', 